In [19]:
import tensorflow as tf 
import numpy as np 

much_data = np.load('training_np--30--30.npy')
train_data = much_data[:3000]
validation_data = much_data[3001:]

In [9]:
validation_data.shape

(1441, 2)

In [10]:
np.random.shuffle(train_data)
np.random.shuffle(validation_data)

In [11]:

sess = tf.InteractiveSession()
#build a softmax regression model
# shape arg is optional, lets tf automatically catch bugs from inconsistent tensorshapes
x = tf.placeholder(tf.float32, shape=[None, 30, 30], name = "x") # 28x28 pixel image = 784, None indicates frst dimension
y_ = tf.placeholder(tf.float32, shape = [None, 2], name = "y") #, shape = [2 ]) # , shape = [None, 2]) # one hot 10-dimensional vector indictaing which digit class (0-9)


#
###
###### MULTILAYER COMPUTATIONAL GRAPH

# TODO. Beautify tensorbaord by using tf.scope and wrapping names around the weight and bias variables 

def weight_variable(shape, w_name):
    initial = tf.truncated_normal(shape, stddev=0.1)

    return tf.Variable(initial, name = 'NN'+str(w_name)+'/W')

# add bias 0.1 to avoid 'dead neurons' from using ReLU neurons/ activation functions
def bias_variable(shape, b_name):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name = 'NN'+str(b_name)+'/B')

# convolution and pooling. handling the boundaries and stride sizes. This is the vanilla vesion (stride of 1, zero padding, output is same size as input)
# pooling is plain old max pooling over 2x2 blocks
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')



In [12]:
# first convolutional layer. consists of our first convolution, followed by max pooling
# convolution will compute 32 features for each 5x5 patch
# weight tensor wil thus have shape of [5, 5, 1, 32] i.e. [ patch size, patch size, input channels, output channels]
with tf.variable_scope('ConvLayer1'):
    W_conv1 = weight_variable([5, 5, 1, 32], 1)
    b_conv1 = bias_variable([32], 1) # bias vector

# reshape x to a 4d tensor, with the 2nd and 3rd dimensions corresponding to image width and height, last dimension is # of color channels
x_image = tf.reshape(x, [-1, 30, 30, 1])

# convolve x_image with the weight tesnor, add bias, apply ReLU function, then max pool
# max pool_2x2 resizes image size to 14x14

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

# second convolutional layer 
# will have 64 features for each 5x5 patch
# with tf.variable_scope('ConvLayer2'):

W_conv2 = weight_variable([5, 5, 32, 64], 2)
b_conv2 = bias_variable([64], 2)

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)


# densely connected layer. Image size has been reduced to 7x7, we add a full_connected layer with 1024 neurons to allow processing on the entire image
# reshape the tensor from the pooling layer into a batch of vectors, multiple by a weight matrix, add bias, apply ReLU
# with tf.variable_scope('DenseConnLayer'):
W_fc1 = weight_variable([8 * 8 * 64, 1024], 'fc1')
b_fc1 = bias_variable([1024], 'fc1')

# with tf.variable_scope('ReshapeImg'):
h_pool2_flat = tf.reshape(h_pool2, [-1, 8*8*64])
h_fc1 = tf.nn.tanh(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# add dropout to reduce overfitting - before readout layer. Dropout only really useful in large CNNs
# turn dropout on during training, turn it off during testing
# with tf.variable_scope('DropOut'):

keep_prob = tf.placeholder(tf.float32, name='keep_prob')
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


# readout layer
W_fc2 = weight_variable([1024, 2], 'fc2')
b_fc2 = bias_variable([2], 'fc2')


y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2



In [18]:
# train and evalutate the model
# 

keep_prob1 = tf.placeholder(tf.float32)

with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels= y_, logits=y_conv))

with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy) # step size is very sensitive

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar("cost", cross_entropy)
tf.summary.scalar("accuracy", accuracy)

summary_op = tf.summary.merge_all()

logs_path = "logs/"
# train_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/train', sess.graph)

with tf.Session() as sess:
    
    # run session, initialise variables
    sess.run(tf.initialize_all_variables())
    
    # writes to TensorBoard 
    writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    batch_size = 50
    
    for k in range(1000):
        
        if k % batch_size == 0 :
    
            d = train_data[k: k+batch_size]
            X, Y = zip(*d)
            X = np.stack(list(X), axis=0)
            Y = np.vstack(list(Y))
            X = X.astype(float)
            Y = Y.astype(float)
            a = np.transpose(Y)
            c = list()
#             c = np.array()

            for i in Y:
                # t=0
                if i == 1:
                    c.append(np.array([0.0,1.0]))
                # t=23 
                if i == 0:
                    c.append(np.array([1.0,0.0]))

            Y = np.vstack(c)
            
            train_accuracy = accuracy.eval(feed_dict={x: X, y_: Y, keep_prob: 1.0})
            
            print("step %d, training accuracy %g" % (k, train_accuracy))
            
        optimizer.run(feed_dict={x : X, y_ : Y, keep_prob:0.9})

    # Creating the Validation Data to validate the model with (and see if it its overfitting)
    d = validation_data
    X, Y = zip(*d)
    X = np.stack(list(X), axis=0)
    Y = np.vstack(list(Y))
    X = X.astype(float)
    Y = Y.astype(float)
    a = np.transpose(Y)
    c = list()
#     c = np.array()
    
    for i in Y: 
        if i == 1:
            c.append(np.array([0.0,1.0]))
        if i == 0:
            c.append(np.array([1.0,0.0]))

    Y = np.vstack(c)
    

    print('test accuracy %g' % accuracy.eval(feed_dict={x: X, y_: Y, keep_prob: 1.0 }))
    
    prediction=sess.run(y_conv ,feed_dict={x: X, keep_prob: 1.0})
#     preds = prediction.eval()
    
#     for i in range(len(preds)):
#         print preds[i]


Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, training accuracy 0.24
step 50, training accuracy 0.72
step 100, training accuracy 0.82
step 150, training accuracy 0.72
step 200, training accuracy 0.82
step 250, training accuracy 0.68
step 300, training accuracy 0.74
step 350, training accuracy 0.74
step 400, training accuracy 0.78
step 450, training accuracy 0.76
step 500, training accuracy 0.86
step 550, training accuracy 0.68
step 600, training accuracy 0.76
step 650, training accuracy 0.86
step 700, training accuracy 0.76
step 750, training accuracy 0.68
step 800, training accuracy 0.62
step 850, training accuracy 0.72
step 900, training accuracy 0.74
step 950, training accuracy 0.68
test accuracy 0.748092


In [22]:
print X[0]

[[ 128.  129.  129.  128.  128.  128.  128.  128.  128.  130.  127.  125.
   126.  129.  136.  144.  151.  144.  134.  133.  142.  149.  164.  168.
   159.  136.  129.  126.  126.  127.]
 [ 127.  129.  129.  128.  128.  128.  126.  127.  129.  130.  131.  134.
   137.  135.  142.  154.  161.  148.  131.  128.  138.  151.  161.  167.
   160.  140.  133.  129.  126.  127.]
 [ 126.  128.  129.  129.  129.  127.  124.  124.  130.  132.  138.  148.
   154.  146.  153.  168.  175.  153.  126.  118.  128.  151.  157.  165.
   161.  143.  139.  133.  128.  126.]
 [ 127.  127.  127.  127.  130.  128.  127.  125.  129.  135.  143.  151.
   156.  151.  150.  157.  159.  137.  111.   99.  105.  127.  160.  171.
   159.  137.  138.  139.  135.  126.]
 [ 128.  127.  126.  126.  130.  130.  130.  128.  128.  139.  142.  141.
   142.  146.  142.  140.  137.  121.  100.   82.   78.  101.  158.  167.
   143.  122.  136.  146.  141.  127.]
 [ 129.  128.  128.  128.  129.  129.  130.  131.  134.  139.  12

In [ ]:
print(len(prediction))

In [91]:
# train and evalutate the model
# 

keep_prob1 = tf.placeholder(tf.float32)

with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels= y_, logits=y_conv))

with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy) # step size is very sensitive

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.scalar_summary("cost", cross_entropy)
tf.scalar_summary("accuracy", accuracy)

summary_op = tf.merge_all_summaries()

epochs = 20

epoch_loss = 0 

logs_path = "logs/"
# train_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/train', sess.graph)

with tf.Session() as sess:
    v
    # run session, initialise variables
    sess.run(tf.initialize_all_variables())
    
    # writes to TensorBoard 
    writer = tf.train.SummaryWriter(logs_path, graph=tf.get_default_graph())
  
    for epoch in range(epochs):
        
        batch_count = 50
    
    #   for i in range(0, len(train_data), 20):
        for i in range(0, 2001, batch_count):
    
            d = train_data[i: i+50]
            X, Y = zip(*d)
            X = np.stack(list(X), axis=0)
            Y = np.vstack(list(Y))
            X = X.astype(float)
            Y = Y.astype(float)
            a = np.transpose(Y)
            c = list()

            for i in Y:
                # t=0
                if i == 1:
                    c.append(np.array([0.0,1.0]))
                # t=23 
                if i == 0:
                    c.append(np.array([1.0,0.0]))

            Y = np.vstack(c)

            loss_ = sess.run(cross_entropy, feed_dict={x: X , y_ : Y, keep_prob: 1.0})
            accur_ = accuracy.eval(feed_dict={x: X, y_: Y, keep_prob: 1.0})

            print("Loss: ", loss_ , "::: Train acc:", accur_  )
            
            _, c = sess.run([optimizer, cross_entropy], feed_dict={x:X, y_:Y, keep_prob: 1.0})

#BIG TODO            
#             _, summary = sess.run([optimizer, summary_op], feed_dict={x:X, y_:Y keep_prob: 0.9})
            
#             writer.add_summary(summary, epoch * batch_count + i)

    print('Epoch', epoch, 'completed out of',epochs,'loss:', loss_)

    # Creating the Validation Data to validate the model with (and see if it its overfitting)
    d = validation_data
    X, Y = zip(*d)
    X = np.stack(list(X), axis=0)
    Y = np.vstack(list(Y))
    X = X.astype(float)
    Y = Y.astype(float)
    a = np.transpose(Y)
    c = list()
    
    for i in Y: 
        if i == 1:
            c.append(np.array([0.0,1.0]))
        if i == 0:
            c.append(np.array([1.0,0.0]))

    Y = np.vstack(c)
    print('test accuracy %g' % accuracy.eval(feed_dict={x: X, y_: Y, keep_prob: 1.0 }))
    


NameError: name 'v' is not defined